In [1]:
import pandas as pd
from numpy import random as npr
from modules import utils

In [2]:
SEED = 2021
npr.seed(SEED)

DeepCTR-PyTorch version 0.2.7 detected. Your version is 0.2.5.
Use `pip install -U deepctr-torch` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR-Torch/releases/tag/v0.2.7


In [8]:
%%time
df = pd.read_csv("data/rating.csv")
utils.evaluate_autorec(df)

Load data finished. Number of users: 6040 Number of items: 3706
IAutoRec.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2021-08-08 16:00:47.285345: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2021-08-08 16:00:47.304754: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2494090000 Hz
2021-08-08 16:00:47.306716: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f6f7c000b60 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-08-08 16:00:47.306740: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-08-08 16:00:47.311823: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-08 16:00:47.312891: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-08 16:00:47.313935: I tenso

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Network built
